In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [216]:
import pandas as pd
import numpy as np
import pdfplumber
import tabula
import seaborn as sns
from Olympic_PED_use.src import functions as fn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from bs4 import BeautifulSoup
import certifi
import urllib3
import re
from csv import DictReader, DictWriter
import datetime as dt

In [3]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000000)

#### Function to scrape tables and return a dataframe

In [4]:
def rtp_scraper(content):
    a = []
    b = []
    c = []
    d = []
    e = []
    f = []
    g = []
    tables = []
    for table in content:
        tables.append(table)
        for table_ in tables:
            rows = table_.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 7:
                    a.append(cells[0].text)
                    b.append(cells[1].text)
                    c.append(cells[2].text)
                    d.append(cells[3].text)
                    e.append(cells[4].text)
                    f.append(cells[5].text)
                    g.append(cells[6].text)
    df = pd.DataFrame(a, columns=['name'])
    df['discipline'] = b
    df['secondary_discipline'] = c
    df['wa_id'] = d
    df['sex'] = e
    df['sanction'] = f
    df['references'] = g
    
    return df

#### Scraping table from athleticsIntegreity.org that holds all athletes currently in the registered testing pool

In [5]:
contents_list = []
for i in range(1,33):
    url = 'https://www.athleticsintegrity.org/know-the-process/registered-testing-pool/p{}?'.format(i)
    req = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                         ca_certs=certifi.where())
    res = req.request('GET', url)
    soup = BeautifulSoup(res.data, 'html.parser')
    contents = soup.find_all('table', class_='table-striped')
    contents_list.append(contents)

In [446]:
dfs = [rtp_scraper(x) for x in contents_list]

Defining first dataframe to append the rest

In [447]:
len(dfs)

32

In [448]:
rtp_df = dfs[0]

#### Appending the remaining dataframes from the contents_list for complete athlete registered testing pool dataframe

In [449]:
for x in range(len(dfs)-1):
    rtp_df = rtp_df.append(dfs[x])

In [450]:
rtp_df = rtp_df.reset_index(drop=True)

In [451]:
rtp_df.discipline.value_counts()

Marathon         247
Half Marathon     47
1500m             36
5000m             31
3000m SC          29
20km Walk         29
800m              29
100m              27
Hammer            26
200m              26
400m              26
400m Hurdles      24
100m Hurdles      23
Shot Put          23
Javelin           23
High Jump         20
Discus            20
Pole Vault        19
10000m            19
Long Jump         18
Triple Jump       18
50km Walk         16
Decathlon         12
Heptathlon        12
Name: discipline, dtype: int64

In [425]:
rtp_df.wa_id

0     14366482
1     14465270
2     14686631
3     14212221
4     14375720
5     14752208
6     14214214
7     14522077
8     14335856
9     14785951
10    14208543
11    14181547
12    14448201
13    14657344
14    14496527
15    14427998
16    14217000
17    14473679
18    14413306
19    14428737
20    14289007
21    14411176
22    14417763
23    14629292
24    14751121
25    14513998
26    14264580
27    14433437
28    14201842
29    14184392
30    14594967
31    14465502
32    14334837
33    14719623
34    14436431
35    14594836
36    14435284
37    14206022
38    14580914
39    14290915
40    14771299
41    14420009
42    14731354
43    14167401
44    14414524
45    14264987
46    14521594
47    14376449
48    14376349
49    14810299
Name: wa_id, dtype: object

In [453]:
rtp_df = rtp_df.sort_values(by='name')

In [454]:
rtp_df_ids = list(rtp_df.wa_id)

In [455]:
len(rtp_df)

800

In [428]:
rtp_df

,name,discipline,secondary_discipline,wa_id,sex,nationality,dob
0,Aaron BROWN,200m,Sprints (400m or less),14366482,M,CAN,1992-05-27
1,Ababel YESHANEH,Marathon,Long Distance (3000m+),14465270,F,ETH,1991-07-22
2,Abdalelah HAROUN,400m,Sprints (400m or less),14686631,M,QAT,1997-01-01
3,Abdelaati IGUIDER,1500m,Middle Distance (800m-1500m),14212221,M,MAR,1987-03-25
4,Abdelmalik LAHOULOU,400m Hurdles,Sprints (400m or less),14375720,M,ALG,1992-05-07
5,Abderrahman SAMBA,400m Hurdles,Sprints (400m or less),14752208,M,QAT,1995-09-05
6,Abdi NAGEEYE,Marathon,Long Distance (3000m+),14214214,M,NED,1989-03-02
7,Abeba-Tekulu GEBREMESKEL,Marathon,Long Distance (3000m+),14522077,F,ETH,1905-06-11
8,Abebe Negewo DEGEFA,Marathon,Long Distance (3000m+),14335856,M,ETH,1984-05-20
9,Abel KIPCHUMBA,Half Marathon,Long Distance (3000m+),14785951,M,KEN,1994-02-03


#### Creating doping_dfs variable to hold the dataframes scraped from the updated july-2020 sanctions list PDF file

In [53]:
doping_dfs = tabula.read_pdf('../data/July-2020-Sanctions-List-Full.pdf', pages='all',
                     output_format='dataframe')

#### Defining first dataframe to append the rest

In [174]:
doping = doping_dfs[0]

In [175]:
for x in range(len(doping_dfs)-1):
    doping = doping.append(doping_dfs[x])

In [176]:
doping.columns = ['name', 'dob', 'nationality', 'roll',
                 'sex', 'discipline_1', 'discipline_2', 
                 'date_of_infraction', 'sanction', 
                 'ineligibility_until', 'dq_date', 
                 'lifetime_ban', 'infraction_type', 
                 'adrv_rules', 'adrv_notes', 'description']

#### Subsetting dataframe with only records of doping cases. (this only removes one athlete record)

In [177]:
doping = doping[doping['infraction_type'] == 'Doping']

#### Converting the infraction_type column values to numeric

In [178]:
numeric_doping = {'Doping':1}

doping.infraction_type = doping.infraction_type.replace(numeric_doping)

#### Renaming this column 'flagged'

In [179]:
doping = doping.rename(columns={'infraction_type':'flagged'})

In [180]:
doping.columns

Index(['name', 'dob', 'nationality', 'roll', 'sex', 'discipline_1', 'discipline_2', 'date_of_infraction', 'sanction', 'ineligibility_until', 'dq_date', 'lifetime_ban', 'flagged', 'adrv_rules', 'adrv_notes', 'description'], dtype='object')

#### Formatting the name values in both dataframes


In [181]:
rtp_df.name = [x.title() for x in df.name]

In [182]:
doping.name = [x.title() for x in doping.name]

#### Formating the names in the doping dataframe to be only the first and last name.

In [68]:
def format_doping_names(df):
    names = []
    for x in df.name:
        split = x.split()
        x = split[-1] + ' ' + split[0]
        names.append(x)
    df.name = [x for x in names]
    return df

In [69]:
doping = format_doping_names(doping)

#### The values in the disqualification date column are not consistent. I need to remove the strings before the dates as well as replace the (.) and (-) with (/). I am also going to convert the dates to only show the year of the disqualification

In [183]:
doping.dq_date.head()

0      Since 08.03.19
1    Since 01-01-2014
2      Since 03.06.18
3      Since 24.08.18
4    Since 24-07-2015
Name: dq_date, dtype: object

In [184]:
doping.dq_date = doping.dq_date.fillna(value='0')
doping.dq_date = [x.strip('Since') for x in doping.dq_date]
doping.dq_date = [x.strip('From') for x in doping.dq_date]

In [185]:
doping = doping.drop([33,41,72,85,94,106,111,121,122,123,124,126,148,149,
                     160,186,204,214,297,305,315,332,351,352,363,373,375,
                     379,381,385,392,400,402,409,411,418,436,437,446,472,
                     483,484,485,489,490,500,505], axis=0)

In [186]:
doping.reset_index(drop=True, inplace=True)

In [187]:
doping.dq_date = [str(x) for x in doping.dq_date]

In [188]:
doping.dq_date = [x.replace('.', '/') for x in doping.dq_date]
doping.dq_date = [x.replace('-', '/') for x in doping.dq_date]

In [189]:
dates = []
for x in doping.dq_date:
    dates.append('20'+x[-2:])
doping.dq_date = [x for x in dates]

In [394]:
doping = doping.sort_values(by='name')

In [396]:
len(doping)

470

In [417]:
doping

,name,dob,nationality,roll,sex,discipline_1,discipline_2,date_of_infraction,sanction,ineligibility_until,dq_date,lifetime_ban,flagged,adrv_rules,adrv_notes,description
0,"Abdul Shahid (Nasera), Haidar",13/01/1981,IRQ,athlete,M,NaN,Throws,08/03/2019,4 years ineligibility,05/05/2023,2019,No,1,"Presence,Use",Metandienone,"In competition test, Iraqi Championships, Bagh..."
1,"Acherki, Mounir",09/02/1981,FRA,athlete,M,1500m,Middle Distance\r(800m-1500m),01/01/2014,4 years ineligibility,15/04/2021,2014,No,1,"Use,Possession",Use or Attempted Use by an Athlete of a\rProhi...,IAAF Rule 32.2(b) Use of a prohibited substanc...
2,"Adamchuk, Mariya",29/05/2000,UKR,athlete,F,Long Jump,Jumps,03/06/2018,4 years ineligibility,16/08/2022,2018,No,1,"Presence,Use","Trenbolone, DMBA &\rMethylhexaneamine","ICT, Ukrainian club U20 Championships', Lutsk,..."
3,"Adekoya, Kemi",16/01/1993,BRN,athlete,F,400m Hurdles,Sprints (400m or\rless),24/08/2018,4 years ineligibility,25/11/2022,2018,No,1,"Presence,Use",Stanozolol,"Out-of-competition test, Jakarta, IDN"
4,"Adeloye, Tosin",07/02/1996,NGR,athlete,F,400m,Sprints (400m or\rless),24/07/2015,8 years ineligibility\r(2nd ADRV),23/07/2023,2015,No,1,"Presence,Use",Exogenous Steroids,"In competition test, Warri Relays - CAA Super..."
5,"Adli Saiful, Muh. Syazrul",17/02/2001,MAS,athlete,M,NaN,Sprints (400m or\rless),18/08/2019,4 years ineligibility,12/12/2023,2019,No,1,"Presence,Use",Methasterone & Oxymetholone,"In competition test, 96th Malaysian Athletics ..."
6,"Adok, Roland",29/10/1984,HUN,athlete,M,NaN,Long Distance\r(3000m+),01/05/2019,2 years ineligibility,22/05/2021,2019,No,1,"Presence,Use",Clenbuterol,"ICT, Hungary 50km ultra running Championships ..."
7,"Agafonova Khanafeyeva,\rGulfiya",NaN,RUS,athlete,F,Hammer,Throws,NaN,8 years ineligibility,05/01/2025,2014,No,1,"Presence,Use",Dehydrochlormethyltestosterone; Use,"(Retesting) In competition test, XXX Olympic G..."
8,"Ahmed, Belal",20/11/1986,EGY,athlete,M,NaN,Long Distance\r(3000m+),27/09/2019,4 years ineligibility,03/12/2023,2019,No,1,"Presence,Use",EPO,"Out-of-competition test, Berlin, GER"
9,"Akkoyun, Mehmet",28/02/1989,TUR,athlete,M,NaN,Long Distance\r(3000m+),11/08/2014,4 years ineligibility,07/02/2021,2014,No,1,Use,Athlete Biological Passport,Athlete Biological Passport Case


# 

# 

# 

#### Loading in the athlete_events.csv file containing the athletes and their statistics

In [386]:
athlete_df = pd.read_csv('../data/athlete_events.csv')

In [387]:
athlete_df.columns = [x.lower() for x in athlete_df.columns]

In [388]:
athlete_df.columns

Index(['id', 'name', 'sex', 'age', 'height', 'weight', 'team', 'noc', 'games', 'year', 'season', 'city', 'sport', 'event', 'medal'], dtype='object')

In [389]:
athletes = athlete_df[(athlete_df.year > 2003) & (athlete_df.year < 2018)]

In [390]:
athlete_names = list(athletes.name.unique())

In [83]:
doping.name = [x.strip(',')for x in doping.name]

In [457]:
athletes.id.value_counts()

13029    10
17755    10
34551    10
44571     8
39035     8
         ..
96584     1
85685     1
16924     1
79554     1
36866     1
Name: id, Length: 6082, dtype: int64

In [84]:
doping_athletes = list(doping.name.unique())

In [391]:
athletes = athletes.sort_values(by='name')

In [392]:
athletes = athletes[athletes['sport'] == 'Athletics']

In [393]:
athletes.city.value_counts()

Rio de Janeiro    2508
London            2278
Beijing           2244
Athina            2175
Name: city, dtype: int64

In [385]:
len(athletes)

9205

In [85]:
len(doping_athletes)

469

In [86]:
len(athlete_names)

37100

In [353]:
for x in range(len(doping_athletes)):
    if doping_athletes[x] in (athlete_names):
        print(doping_athletes[x])

Jordan Chipangama
Adrian Griffith
Ruth Jebet
Manpreet Kaur
Dinesh Kumar
Nigel Levine
Kabange Mupopo
Tezdzhan Naimova
Aziz Ouhadi
Athanasia Perra
Alex Schwazer
Avtar Singh
Baljinder Singh
Binnaz Uslu


In [88]:
len(df)

800

In [192]:
elite_athlete_names = list(df.name)

In [193]:
matched = []
for x in range(len(elite_athlete_names) -1):
    if elite_athlete_names[x] in (doping_athletes):
        matched.append(elite_athlete_names[x])

In [194]:
len(matched)

1

In [92]:
elite_athlete_names

['Aaron Brown',
 'Ababel Yeshaneh',
 'Abdalelah Haroun',
 'Abdelaati Iguider',
 'Abdelmalik Lahoulou',
 'Abderrahman Samba',
 'Abdi Nageeye',
 'Abeba-Tekulu Gebremeskel',
 'Abebe Negewo Degefa',
 'Abel Kipchumba',
 'Abel Kirui',
 'Abera Kuma',
 'Aberu Mekuria Zennebe',
 'Abraham Kibiwot',
 'Abrha Milaw Asefa',
 'Adam Gemili',
 'Adam Kszczot',
 'Adugna Takele',
 'Afera Godfay',
 'Afewerk Mesfin',
 'Agnes Jebet Tirop',
 'Ajee Wilson',
 'Akani Simbine',
 'Akeem Bloomfield',
 'Albert Chemutai',
 'Aaron Brown',
 'Ababel Yeshaneh',
 'Abdalelah Haroun',
 'Abdelaati Iguider',
 'Abdelmalik Lahoulou',
 'Abderrahman Samba',
 'Abdi Nageeye',
 'Abeba-Tekulu Gebremeskel',
 'Abebe Negewo Degefa',
 'Abel Kipchumba',
 'Abel Kirui',
 'Abera Kuma',
 'Aberu Mekuria Zennebe',
 'Abraham Kibiwot',
 'Abrha Milaw Asefa',
 'Adam Gemili',
 'Adam Kszczot',
 'Adugna Takele',
 'Afera Godfay',
 'Afewerk Mesfin',
 'Agnes Jebet Tirop',
 'Ajee Wilson',
 'Akani Simbine',
 'Akeem Bloomfield',
 'Albert Chemutai',
 'Albert

In [93]:
def wiki_scraper(content):
    a = []
    b = []
    c = []
    d = []
    e = []
    f = []
    g = []
    tables = []
    for table in content:
        tables.append(table)
        for table_ in tables:
            rows = table_.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 7:
                    a.append(cells[0].text)
                    b.append(cells[1].text)
                    c.append(cells[2].text)
                    d.append(cells[3].text)
                    e.append(cells[4].text)
                    f.append(cells[5].text)
                    g.append(cells[6].text)
    df = pd.DataFrame(a, columns=['name'])
    df['country'] = b
    df['event'] = c
    df['date_of_violation'] = d
    df['substance'] = e
    df['sanction'] = f
    df['references'] = g
    
    return df

In [94]:
url = 'https://en.wikipedia.org/wiki/List_of_doping_cases_in_athletics'
req = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                         ca_certs=certifi.where())
res = req.request('GET', url)
soup = BeautifulSoup(res.data, 'html.parser')
contents = soup.find_all('table', class_='wikitable sortable')  

In [292]:
wiki_doping = wiki_scraper(contents)

In [293]:
def col_format(df):
    df['name'] = [x.strip('\n') for x in df['name']]
    df['country'] = [x.strip('\n') for x in df['country'].values]
    df['date_of_violation'] = [x.strip('\n') for x in df['date_of_violation'].values]
    df['event'] = [x.strip('\n') for x in df['event'].values]
    df['substance'] = [x.strip('\n') for x in df['substance'].values]
    df['sanction'] = [x.strip('\n') for x in df['sanction'].values]
    df['references'] = [x.strip('\n') for x in df['references'].values]
    return df

In [294]:
wiki_doping = col_format(wiki_doping)
wiki_doping

,name,country,event,date_of_violation,substance,sanction,references
0,Nunu Abashidze,Soviet Union,Shot put,1981,,,[1][2]
1,Ahmed Abd El Raouf,Egypt,Hammer throw,2008,Norandrosterone,2 years,[3][4]
2,Inga Abitova,Russia,Long distance,2009,Biological passport anomalies,2 years,[5][6][7]
3,Folashade Abugan,Nigeria,Sprinting,2010,Testosterone prohormone,2 years,[8][9][10]
4,Ibrahim Mohamed Aden,Somalia,Middle distance,1999,Ephedrine,Public warning,[11][12][13]
...,...,...,...,...,...,...,...
14482,Zhou Wei,China,Sprinting,2000,,2 years,[264]
14483,Zohar Zimro,Israel,Marathon,2012,,Public reprimand,[219]
14484,Yevgeniya Zinurova,Russia,Middle distance,2010,Biological passport abnormalities,2 years,[516]
14485,Khalid Zoubaa,France,Long distance,2007,EPO,3 years,[887]


In [295]:
wiki_doping.date_of_violation = [x.replace('23 May 2002  24 June 2002', '2002') for x in wiki_doping.date_of_violation]
wiki_doping.date_of_violation = [x[:4] for x in wiki_doping.date_of_violation]

In [296]:
drop_value = wiki_doping.date_of_violation[44]

In [297]:
wiki_doping = wiki_doping[wiki_doping.date_of_violation != drop_value]

In [298]:
wiki_doping = wiki_doping[wiki_doping.date_of_violation != 'Unkn']

In [299]:
len(wiki_doping)

14169

In [300]:
rows_to_drop = [0, 60, 197, 390, 632, 905, 1198, 1532, 1874, 1907, 2249,
                2293, 2635, 2697, 3039, 3173, 3515, 3693, 4035, 4278, 4289,
                4631, 4874, 4902, 5244, 5487, 5536, 5878, 6121, 6196, 6212,
                6554, 6797, 6872, 6889, 7231, 7474, 7549, 7603, 7945, 8188,
                8263, 8392, 8734, 8977, 9052, 9219, 9561, 9804, 9879, 10052,
                10394, 10637, 10712, 10907, 11249, 11492, 11567, 11789, 12131,
                12374, 12449, 12672, 13014, 13257, 13332, 13572, 13914, 14157, 14232]

In [301]:
wiki_doping = wiki_doping.drop(rows_to_drop, axis=0)

In [302]:
wiki_doping.date_of_violation = [int(x) for x in wiki_doping.date_of_violation]

In [284]:
wiki_doping = wiki_doping[(wiki_doping.date_of_violation > 2003) & (wiki_doping.date_of_violation < 2017)]

In [285]:
wiki_doping.duplicated().sum()

8595

In [286]:
len(wiki_doping)

9174

,name,country,event,date_of_violation,substance,sanction,references
1,Ahmed Abd El Raouf,Egypt,Hammer throw,2008,Norandrosterone,2 years,[3][4]
2,Inga Abitova,Russia,Long distance,2009,Biological passport anomalies,2 years,[5][6][7]
3,Folashade Abugan,Nigeria,Sprinting,2010,Testosterone prohormone,2 years,[8][9][10]
4,Ibrahim Mohamed Aden,Somalia,Middle distance,1999,Ephedrine,Public warning,[11][12][13]
5,Tosin Adeloye,Nigeria,Sprinting,2012,Metenolone Exogenous steroids,2 years 8 years,[14][15][16][17]
6,Funke Adeoye,Nigeria,Hammer throw,2007,Ephedrine,Public warning,[3][18][19][20]
7,Alessandra Aguilar,Spain,Long distance,2011,,3 months,[21]
8,Amina Aït Hammou,Morocco,Middle distance,2008,3 whereabouts failures,1 year,[3][22]
9,Seltana Aït Hammou,Morocco,Middle distance,2008,3 whereabouts failures,1 year,[3][23]
10,Chioma Ajunwa,Nigeria,"Long jump, sprinting",1992,,4 years,[24][25]


In [339]:
wiki_doping = wiki_doping.drop_duplicates()

In [340]:
len(wiki_doping)

893

In [341]:
wiki_names = list(wiki_doping.name.unique())

In [346]:
matched = []
for x in range(len(athlete_names) -1):
    if athlete_names[x] in (wiki_names):
        matched.append(athlete_names[x])

In [348]:
len(matched)

92

In [345]:
athlete_names.sort()

In [208]:
athlete_names

['  Gabrielle Marie "Gabby" Adcock (White-)',
 ' Th Anh',
 ' Th Ngn Thng',
 'A Lamusi',
 'A. Joshua "Josh" West',
 'Aadam Ismaeel Khamis',
 'Aagje Vanwalleghem',
 'Aarik Wilson',
 'Aarn Galindo Rubio',
 'Aarn Sarmiento Padilla',
 'Aaron Arthur Cook',
 'Aaron Blunck',
 'Aaron Brown',
 'Aaron Cleare',
 'Aaron Duane Olson',
 'Aaron Egbele',
 'Aaron Feltham',
 'Aaron Gate',
 'Aaron J. "AJ" Bear',
 'Aaron James Ramsey',
 'Aaron James Scott',
 'Aaron John Royle',
 'Aaron Kenneth Myette',
 'Aaron Lowe',
 'Aaron March',
 'Aaron Nigel Armstrong',
 'Aaron Parchem',
 'Aaron Russell',
 'Aaron Wells Peirsol',
 'Aaron Younger',
 'Aas Mmmdov',
 'Aauri Lorena Bokesa Abia',
 'Ababel Yeshaneh Birhane',
 'Abadi Hadis Embaye',
 'Abbas Al-Qaisoum',
 'Abbas Dabbaghi Souraki',
 'Abbas Fallah',
 'Abbas Qali',
 'Abbas Saeidi Tanha',
 'Abbas Samimi',
 'Abbey Weitzeil',
 'Abbos Atayev',
 'Abbos Rakhmonov',
 'Abbubaker Mobara',
 'Abby Bishop',
 'Abby May Erceg',
 'Abdal Salam Al-Dabaji',
 'Abdalelah Haroun Hassan

In [344]:
wiki_names.sort()

In [214]:
len(athlete_names)

37100

In [210]:
for x in list(zip(athlete_names, wiki_names)):
    print(x)

('  Gabrielle Marie "Gabby" Adcock (White-)', 'Abdelatif Chemlal')
(' Th Anh', 'Abdelhadi Habassa')
(' Th Ngn Thng', 'Abdelhadi Labäli')
('A Lamusi', 'Abdelkader Hachlaf')
('A. Joshua "Josh" West', 'Abdellah Haidane')
('Aadam Ismaeel Khamis', 'Abderahmane Djemadi')
('Aagje Vanwalleghem', 'Abderrahim El Asri')
('Aarik Wilson', 'Abderrahim Goumri')
('Aarn Galindo Rubio', 'Abderrahime Bouramdane')
('Aarn Sarmiento Padilla', 'Abdulagadir Idriss')
('Aaron Arthur Cook', 'Abraham Kiprotich')
('Aaron Blunck', 'Abubaker Ali Kamal')
('Aaron Brown', 'Adil Kaouch')
('Aaron Cleare', 'Adrián Annus')
('Aaron Duane Olson', 'Adriënne Herzog')
('Aaron Egbele', 'Agatha Jeruto Kimaswai')
('Aaron Feltham', 'Agnieszka Gortel-Maciuk')
('Aaron Gate', 'Agustín Félix')
('Aaron J. "AJ" Bear', 'Aham Okeke')
('Aaron James Ramsey', 'Ahmad Hazer')
('Aaron James Scott', 'Ahmed Abd El Raouf')
('Aaron John Royle', 'Ahmed Baday')
('Aaron Kenneth Myette', 'Ahmed Faiz')
('Aaron Lowe', 'Ahmed Mohamed Dheeb')
('Aaron March'

In [356]:
wiki_doping = wiki_doping.sort_values(by='name')

In [360]:
wiki_doping = wiki_doping[wiki_doping.date_of_violation > 2003]

In [363]:
wiki_doping = wiki_doping[wiki_doping.date_of_violation < 2017]

In [364]:
wiki_doping.date_of_violation.value_counts()

2011    67
2009    64
2012    59
2013    57
2010    51
2008    50
2014    47
2004    42
2007    39
2005    34
2006    32
2015    26
2016    11
Name: date_of_violation, dtype: int64

In [460]:
import glob

In [468]:
def create_athlete_df():
   
    dfs = []
    csvs = glob.glob('../data/athletes_by_event/*')
    for csv in csvs:
        #print('loading_csv:{}'.format(csv))
        df_ = pd.read_csv(csv, delimiter='\t', keep_default_na=False)
        df_ = df_.drop('Unnamed: 5', axis=1)
        dfs.append(df_)
    if len(dfs) == 40:
        print('finished')
    df = dfs[0]
    for x in range(len(dfs)):
        
        df = df.append(dfs[x])
    
    df.columns = [x.lower() for x in df.columns]
    df.name = [x.title() for x in df.name]
    
    df = df.sort_values(by='name')
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    
    return df

In [469]:
athlete_df = create_athlete_df()

finished


In [470]:
athlete_df.tail()

,id,name,nation(s),sport(s),era
5129,120281,Žana Jereb,SLO,ATH,2012
5130,105404,Živilė Balčiünaité,LTU,ATH,2004—2008
5131,131009,Živilė Vaiciukevičiūtė,LTU,ATH,2016
5132,106600,Əlibəy Şükürov,AZE,ATH,2004
5133,105625,Ștefan Vasilache,ROU,ATH,2004


In [466]:
athlete_df.name

0                          A. G. Kruger
1                          Aadam Khamis
2                          Aarik Wilson
3                       Aaron Armstrong
4                           Aaron Brown
5                           Aaron Brown
6                          Aauri Bokesa
7                       Ababel Yeshaneh
8                           Abadi Hadis
9                          Abbas Samimi
10                     Abbey D'Agostino
11                Abdal Salam Al-Dabaji
12                     Abdalelah Haroun
13                   Abdalla Abdelgadir
14                       Abdalla Yousif
15                 Abdel Aziz Merzougui
16                   Abdelati El-Guesse
17                    Abdelatif Chemlal
18          Abdelaziz Ennaji El-Idrissi
19                     Abdelhak Zakaria
20                   Abdelhakim Maazouz
21                   Abdelkader Hachlaf
22                       Abdellah Falil
23                    Abdellatif Meftah
24                Abdelmajid El-Hissouf
